In [ ]:
#エントロピーを利用した特徴選択とFTTransformerによる心疾患予測
#このコードは、心疾患予測のためのFTTransformerモデルを使用し
#エントロピーを利用して特徴選択を行うものです。
#モデルは5つのfoldで学習され、各foldごとに推論を行い、
#改善された予測結果をCSVファイルに保存します。
#5に加えて，CSVに各項目の期待値を出力するように改良
import os
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay,
    precision_score, recall_score, f1_score
)
import matplotlib.pyplot as plt
from tqdm import tqdm
from math import log
from fttransformer_mask import FTTransformer

# ============ 設定 ============
DATA_PATH = "cardio_train.csv"
TEST_RATIO = 0.2
categorical_features = ["gender", "cholesterol", "gluc", "smoke", "alco", "active"]
continuous_features = ["age", "height", "weight", "ap_hi", "ap_lo"]
additional_features = ["cholesterol", "gluc", "smoke", "alco", "active"]
MASK_BASE = [False, False] + [True] * len(additional_features) + [False] * len(continuous_features)
DEVICE = torch.device("cpu")  # GPU利用時は "cuda" に変更

# エントロピー計算関数
def entropy(p: float) -> float:
    if p <= 0 or p >= 1:
        return 0.0
    return - (p * log(p) + (1 - p) * log(1 - p))

# モデル保存パス
fold_to_model_path = {
    1: "./save_mask_models/best_model_epoch300_fold1_ValAcc0.732.pth",
    2: "./save_mask_models/best_model_epoch300_fold2_ValAcc0.739.pth",
    3: "./save_mask_models/best_model_epoch300_fold3_ValAcc0.731.pth",
    4: "./save_mask_models/best_model_epoch300_fold4_ValAcc0.737.pth",
    5: "./save_mask_models/best_model_epoch300_fold5_ValAcc0.735.pth",
}

# ============ メイン処理 ============
def main():
    # データ読み込み＆分割
    data = pd.read_csv(DATA_PATH, sep=';')
    ids = data["id"].values
    X = data.drop(columns=["cardio"])
    y = data["cardio"]
    X_trainval, X_test, y_trainval, y_test, id_trainval, id_test = train_test_split(
        X, y, ids, test_size=TEST_RATIO, random_state=42, stratify=y
    )

    # カテゴリ変数エンコード
    encoders = {}
    for col in categorical_features:
        le = LabelEncoder()
        X_trainval[col] = le.fit_transform(X_trainval[col])
        X_test[col] = le.transform(X_test[col])
        encoders[col] = le

    # 連続値スケーリング
    scaler = StandardScaler()
    X_trainval[continuous_features] = scaler.fit_transform(X_trainval[continuous_features])
    X_test[continuous_features] = scaler.transform(X_test[continuous_features])

    # 出現確率再計算
    feature_value_probs = {
        feat: X_trainval[feat].value_counts(normalize=True).sort_index().to_dict()
        for feat in additional_features
    }

    # 各foldごとに処理
    for fold, model_path in fold_to_model_path.items():
        print(f"=== Fold{fold} モデル読み込み: {model_path} ===")
        if not os.path.exists(model_path):
            print(f"❌ モデルが存在しません: {model_path}")
            continue

        # モデルロード
        model = FTTransformer(
            categories=[X[cat].nunique() for cat in categorical_features],
            num_continuous=len(continuous_features),
            dim=64, depth=6, heads=8, ff_dropout=0.2, attn_dropout=0.2
        ).to(DEVICE)
        model.load_state_dict(torch.load(model_path, map_location=DEVICE))
        model.eval()

        results = []

        # 推論ループ（進捗バー付き）
        for i in tqdm(range(len(X_test)), desc=f"Fold{fold} 推論", unit="件"):
            sid = int(id_test[i])
            label = int(y_test.iloc[i])
            x_cat = torch.tensor([X_test.iloc[i][categorical_features].values], dtype=torch.long).to(DEVICE)
            x_cont = torch.tensor([X_test.iloc[i][continuous_features].values.astype(np.float32)], dtype=torch.float32).to(DEVICE)

            # ベース推論 + エントロピー
            with torch.no_grad():
                base_conf = torch.sigmoid(
                    model(x_cat, x_cont, torch.tensor([MASK_BASE], dtype=torch.bool).to(DEVICE))
                ).item()
            base_ent = entropy(base_conf)
            base_pred = int(base_conf > 0.5)
            was_correct_before = (base_pred == label)

            # エントロピー削減量で特徴選択と期待エントロピー取得
            expected_ent_map = {}
            reduction_map = {}
            best_reduction = -1.0
            best_feat = None
            for feat in additional_features:
                mask = MASK_BASE.copy()
                mask[2 + additional_features.index(feat)] = False
                expected_ent = 0.0
                for val, prob in feature_value_probs[feat].items():
                    x_alt = x_cat.clone()
                    x_alt[0, categorical_features.index(feat)] = int(val)
                    with torch.no_grad():
                        conf_alt = torch.sigmoid(
                            model(x_alt, x_cont, torch.tensor([mask], dtype=torch.bool).to(DEVICE))
                        ).item()
                    expected_ent += entropy(conf_alt) * prob
                expected_ent_map[feat] = expected_ent
                reduction_map[feat] = base_ent - expected_ent
                if reduction_map[feat] > best_reduction:
                    best_reduction = reduction_map[feat]
                    best_feat = feat

            # 改良後推論
            final_mask = MASK_BASE.copy()
            final_mask[2 + additional_features.index(best_feat)] = False
            with torch.no_grad():
                improved_conf = torch.sigmoid(
                    model(x_cat, x_cont, torch.tensor([final_mask], dtype=torch.bool).to(DEVICE))
                ).item()
            imp_pred = int(improved_conf > 0.5)
            was_correct_after = (imp_pred == label)

            # 変化タイプ
            if not was_correct_before and was_correct_after:
                change_type = 'incorrect->correct'
            elif was_correct_before and not was_correct_after:
                change_type = 'correct->incorrect'
            elif was_correct_before and was_correct_after:
                change_type = 'correct->correct'
            else:
                change_type = 'incorrect->incorrect'

            # ランキング (削減量順)
            sorted_feats = sorted(additional_features, key=lambda f: reduction_map[f], reverse=True)
            feat_rank_order = ">".join(sorted_feats)

            # 結果記録
            row = {
                'id': sid,
                'answer': label,
                'chosen_feat': best_feat,
                'base_pred': base_pred,
                'improved_pred': imp_pred,
                'was_correct_before': was_correct_before,
                'was_correct_after': was_correct_after,
                'change_type': change_type,
                'feat_rank_order': feat_rank_order
            }
            for feat in additional_features:
                row[f'expected_ent_{feat}'] = expected_ent_map[feat]
            results.append(row)

        # Accuracy, Precision, Recall, F1計算
        y_true = [r['answer'] for r in results]
        y_base = [r['base_pred'] for r in results]
        y_imp = [r['improved_pred'] for r in results]
        base_acc = sum(r['was_correct_before'] for r in results) / len(results)
        imp_acc = sum(r['was_correct_after'] for r in results) / len(results)
        base_prec = precision_score(y_true, y_base)
        base_rec = recall_score(y_true, y_base)
        base_f1 = f1_score(y_true, y_base)
        imp_prec = precision_score(y_true, y_imp)
        imp_rec = recall_score(y_true, y_imp)
        imp_f1 = f1_score(y_true, y_imp)
        print(
            f"Fold{fold} - base acc: {base_acc:.4f}, prec: {base_prec:.4f}, rec: {base_rec:.4f}, f1: {base_f1:.4f} | "
            f"improved acc: {imp_acc:.4f}, prec: {imp_prec:.4f}, rec: {imp_rec:.4f}, f1: {imp_f1:.4f}"
        )

        # 混同行列を横並びでプロット
        cm_base = confusion_matrix(y_true, y_base)
        cm_imp = confusion_matrix(y_true, y_imp)
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        labels = ['nocardio', 'cardio']
        ConfusionMatrixDisplay(cm_base, display_labels=labels).plot(ax=axes[0], cmap='Blues', colorbar=False)
        axes[0].set_title('Base CM')
        ConfusionMatrixDisplay(cm_imp, display_labels=labels).plot(ax=axes[1], cmap='Blues', colorbar=False)
        axes[1].set_title('Improved CM')
        fig.suptitle(f'Fold{fold} Confusion Matrices')
        fig.text(
            0.5, -0.05,
            f"Base -> acc: {base_acc:.3f}, prec: {base_prec:.3f}, rec: {base_rec:.3f}, f1: {base_f1:.3f} | "
            f"Imp -> acc: {imp_acc:.3f}, prec: {imp_prec:.3f}, rec: {imp_rec:.3f}, f1: {imp_f1:.3f}",
            ha='center'
        )
        plt.tight_layout()
        plt.savefig(f'fold{fold}_cms.png', bbox_inches='tight')
        plt.close()

        # CSV保存
        pd.DataFrame(results).sort_values('id').to_csv(f"fold{fold}_results.csv", index=False)
        print(f"✅ fold{fold}_results.csv saved")

if __name__ == "__main__":
    main()


=== Fold1 モデル読み込み: ./save_mask_models/best_model_epoch300_fold1_ValAcc0.732.pth ===


Fold1 推論:   0%|          | 0/14000 [00:00<?, ?件/s]/tmp/ipykernel_1213669/2490003468.py:89: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  x_cat = torch.tensor([X_test.iloc[i][categorical_features].values], dtype=torch.long).to(DEVICE)
Fold1 推論: 100%|██████████| 14000/14000 [29:41<00:00,  7.86件/s]


Fold1 - base accuracy: 0.6943, improved accuracy: 0.7056
✅ fold1_results.csv saved
=== Fold2 モデル読み込み: ./save_mask_models/best_model_epoch300_fold2_ValAcc0.739.pth ===


Fold2 推論: 100%|██████████| 14000/14000 [30:55<00:00,  7.55件/s]


Fold2 - base accuracy: 0.6929, improved accuracy: 0.7050
✅ fold2_results.csv saved
=== Fold3 モデル読み込み: ./save_mask_models/best_model_epoch300_fold3_ValAcc0.731.pth ===


...

Fold3 - base accuracy: 0.6954, improved accuracy: 0.7054
✅ fold3_results.csv saved
=== Fold4 モデル読み込み: ./save_mask_models/best_model_epoch300_fold4_ValAcc0.737.pth ===


Fold4 推論: 100%|██████████| 14000/14000 [31:05<00:00,  7.50件/s]


Fold4 - base accuracy: 0.6964, improved accuracy: 0.7013
✅ fold4_results.csv saved
=== Fold5 モデル読み込み: ./save_mask_models/best_model_epoch300_fold5_ValAcc0.735.pth ===


Fold5 推論: 100%|██████████| 14000/14000 [31:50<00:00,  7.33件/s]


Fold5 - base accuracy: 0.6899, improved accuracy: 0.7025
✅ fold5_results.csv saved
